## xgb/ lgbm

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime, timedelta
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve, auc
from sklearn.preprocessing import StandardScaler

# Import necessary libraries
import matplotlib.pyplot as plt

# Load the data
arrest_data = pd.read_csv('../data/chicago_clean_final_arrest.csv')
non_arrest_data = pd.read_csv('../data/chicago_clean_final_not_arrest.csv')

# Convert date column to datetime
arrest_data['Date'] = pd.to_datetime(arrest_data['Date'], format='%Y-%m-%d %H:%M:%S')
non_arrest_data['Date'] = pd.to_datetime(non_arrest_data['Date'], format='%Y-%m-%d %H:%M:%S')

# Set date as index if it's not already
if 'Date' in arrest_data.columns:
    arrest_data = arrest_data.set_index('Date')
    non_arrest_data = non_arrest_data.set_index('Date')

# Define our prediction goal
# We will predict whether a specific grid will have more than the median crime rate in the next 24 hours

# Feature Engineering

# 1. Temporal Features
def add_temporal_features(df):
    """Add time-based features to the dataframe."""
    df_temp = df.reset_index()
    
    # Basic time features
    df_temp['hour'] = df_temp['Date'].dt.hour
    df_temp['day_of_week'] = df_temp['Date'].dt.dayofweek
    df_temp['day_of_month'] = df_temp['Date'].dt.day
    df_temp['month'] = df_temp['Date'].dt.month
    df_temp['year'] = df_temp['Date'].dt.year
    df_temp['is_weekend'] = df_temp['day_of_week'].isin([5, 6]).astype(int)
    
    # Time of day categories
    df_temp['time_of_day'] = pd.cut(
        df_temp['hour'], 
        bins=[0, 6, 12, 18, 24], 
        labels=['night', 'morning', 'afternoon', 'evening']
    )
    
    # One-hot encode time of day
    time_of_day_dummies = pd.get_dummies(df_temp['time_of_day'], prefix='time')
    df_temp = pd.concat([df_temp, time_of_day_dummies], axis=1)
    
    # Holidays (US holidays - simplified for demo)
    holidays = [
        # New Year's Day
        pd.Timestamp(year=y, month=1, day=1) for y in range(2001, 2026)
    ] + [
        # Independence Day
        pd.Timestamp(year=y, month=7, day=4) for y in range(2001, 2026)
    ] + [
        # Christmas
        pd.Timestamp(year=y, month=12, day=25) for y in range(2001, 2026)
    ]
    
    df_temp['is_holiday'] = df_temp['Date'].isin(holidays).astype(int)
    
    return df_temp.set_index('Date')

# 2. Spatial Features - Add grid coordinates 
def add_spatial_features(df):
    """Extract spatial features from the grid columns."""
    grid_columns = df.columns
    
    # Create features for each grid
    for grid in grid_columns:
        if '_' not in grid:
            continue
        
        x, y = map(int, grid.split('_'))
        
        # Create grid distance feature (distance from city center, assuming grid 5_5 is downtown)
        center_x, center_y = 5, 5
        df[f'dist_from_center'] = np.sqrt((x - center_x)**2 + (y - center_y)**2)
        
        # Create grid location features
        df[f'is_north'] = int(y > 5)
        df[f'is_south'] = int(y < 5)
        df[f'is_east'] = int(x > 5)
        df[f'is_west'] = int(x < 5)
    
    return df

# 3. Rolling Window Features
def add_rolling_features(df, windows=[1, 3, 6, 12, 24, 48, 168]):
    """Add rolling window statistics as features."""
    df_rolling = df.copy()
    
    for grid in df.columns:
        if '_' not in grid:
            continue
            
        for window in windows:
            # Rolling means
            df_rolling[f'roll_mean_{window}h'] = df[grid].rolling(window=window, min_periods=1).mean()
            
            # Rolling standard deviations (volatility)
            df_rolling[f'roll_std_{window}h'] = df[grid].rolling(window=window, min_periods=1).std().fillna(0)
            
            # Rolling maximum
            df_rolling[f'roll_max_{window}h'] = df[grid].rolling(window=window, min_periods=1).max()
    
    return df_rolling

# 4. Lagged Features
def add_lag_features(df, lags=[1, 3, 6, 12, 24, 48, 168]):
    """Add lagged values as features."""
    df_lag = df.copy()
    
    for grid in df.columns:
        if '_' not in grid:
            continue
            
        for lag in lags:
            df_lag[f'lag_{lag}h'] = df[grid].shift(lag)
    
    # Fill NaN values with 0 (for the beginning of the time series)
    df_lag = df_lag.fillna(0)
    return df_lag

# 5. Interaction Features
def add_interaction_features(df, arrest_df):
    """Add interaction features between grids and arrest rate."""
    # Calculate arrest rate (arrest / (arrest + non-arrest))
    all_crime = arrest_df + df
    arrest_rate = arrest_df / all_crime.replace(0, 1)  # Prevent division by zero
    
    df_interact = df.copy()
    
    for grid in df.columns:
        if '_' not in grid:
            continue
            
        # Add arrest rate feature
        df_interact[f'arrest_rate'] = arrest_rate[grid]
        
        # Add neighbor interaction
        x, y = map(int, grid.split('_'))
        
        # Find neighboring grids
        neighbors = []
        for dx, dy in [(0,1), (1,0), (0,-1), (-1,0)]:  # Up, right, down, left
            nx, ny = x + dx, y + dy
            neighbor = f"{nx}_{ny}"
            if neighbor in df.columns:
                neighbors.append(neighbor)
        
        # If grid has neighbors, create features
        if neighbors:
            # Crime spread from neighbors
            df_interact[f'neighbor_sum'] = df[[n for n in neighbors]].sum(axis=1)
            
            # Arrest rate of neighbors
            neighbor_arrest_rate = arrest_rate[[n for n in neighbors]].mean(axis=1)
            df_interact[f'neighbor_arrest_rate'] = neighbor_arrest_rate
    
    return df_interact

# Function to prepare target variable
def prepare_target(df, threshold_percentile=50, prediction_window=24):
    """
    Create target variable: 1 if crime is above threshold in next window, else 0
    """
    # Calculate future crime counts (sum over next prediction_window hours)
    future_crime = pd.DataFrame(index=df.index)
    
    for grid in df.columns:
        if '_' not in grid:
            continue
            
        # Sum crime counts over next prediction_window hours
        future_crime[grid] = df[grid].rolling(window=prediction_window).sum().shift(-prediction_window)
    
    # Determine threshold for each grid
    thresholds = {}
    for grid in future_crime.columns:
        if '_' not in grid:
            continue
        thresholds[grid] = np.percentile(future_crime[grid].dropna(), threshold_percentile)
    
    # Create binary target: 1 if crime is above threshold, 0 otherwise
    target = pd.DataFrame(index=future_crime.index)
    for grid in future_crime.columns:
        if '_' not in grid:
            continue
        target[f'target'] = (future_crime[grid] > thresholds[grid]).astype(int)
    
    return target

# Create a model training and evaluation pipeline
def train_evaluate_model(X_train, X_test, y_train, y_test, model_type='xgboost'):
    """Train and evaluate model for crime prediction."""
    
    if model_type == 'xgboost':
        model = xgb.XGBClassifier(
            learning_rate=0.1,
            n_estimators=100,
            max_depth=5,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42
        )
    elif model_type == 'lightgbm':
        model = lgb.LGBMClassifier(
            learning_rate=0.1,
            n_estimators=100,
            max_depth=5,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42
        )
    # elif model_type == 'randomforest':
    #     model = RandomForestClassifier(
    #         n_estimators=100,
    #         max_depth=5,
    #         min_samples_split=10,
    #         min_samples_leaf=5,
    #         random_state=42
    #     )
    # elif model_type == 'logistic':
    #     # Scale features for logistic regression
    #     scaler = StandardScaler()
    #     X_train = scaler.fit_transform(X_train)
    #     X_test = scaler.transform(X_test)
        
    #     model = LogisticRegression(
    #         C=0.1,
    #         penalty='l2',
    #         solver='lbfgs',
    #         max_iter=1000,
    #         random_state=42
    #     )
    else:
        raise ValueError("Model type not supported")
    
    # Add this before the model.fit() in your train_evaluate_model function
    print("X_train info:")
    print(X_train.info())
    print("X_train columns with object dtype:")
    print(X_train.select_dtypes(include=['object']).columns.tolist())
    print("X_train sample:")
    print(X_train.head())

    # If object columns exist, convert them
    for col in X_train.select_dtypes(include=['object']).columns:
        X_train[col] = pd.to_numeric(X_train[col], errors='coerce')
        X_test[col] = pd.to_numeric(X_test[col], errors='coerce')

    X_train = X_train.fillna(0)
    X_test = X_test.fillna(0)
    X_train_np = X_train.values.astype(np.float32)
    X_test_np = X_test.values.astype(np.float32)
    
    # Train the model
    model.fit(X_train_np, y_train)
    
    # Evaluate model
    y_pred_proba = model.predict_proba(X_test_np)[:, 1]
    y_pred = model.predict(X_test_np)
    
    # Calculate metrics
    precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
    pr_auc = auc(recall, precision)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    print(f"Model: {model_type}")
    print(f"ROC AUC: {roc_auc:.4f}")
    print(f"PR AUC: {pr_auc:.4f}")
    print(classification_report(y_test, y_pred))
    
    # Feature importance
    if hasattr(model, 'feature_importances_'):
        feature_importance = pd.DataFrame({
            'feature': X_train.columns,
            'importance': model.feature_importances_
        }).sort_values('importance', ascending=False)
        print("Top 10 important features:")
        print(feature_importance.head(10))
    
    return model, y_pred_proba

# This will be our main pipeline function that puts everything together
def crime_prediction_pipeline(grid_to_predict='5_5'):
    """
    Run the complete crime prediction pipeline for a specific grid.
    """
    print(f"Preparing prediction model for grid {grid_to_predict}...")
    
    # 1. Feature Engineering
    print("Adding temporal features...")
    arrest_temp = add_temporal_features(arrest_data)
    non_arrest_temp = add_temporal_features(non_arrest_data)
    
    # Keep only the grid column we want to predict
    arrest_grid = arrest_temp[[grid_to_predict]]
    non_arrest_grid = non_arrest_temp[[grid_to_predict]]
    
    print("Adding rolling and lag features...")
    arrest_features = add_rolling_features(arrest_grid)
    arrest_features = add_lag_features(arrest_features)
    
    non_arrest_features = add_rolling_features(non_arrest_grid)
    non_arrest_features = add_lag_features(non_arrest_features)
    
    # Create target variable
    print("Preparing target variable...")
    target = prepare_target(non_arrest_grid)
    
    # Combine features and ensure same index
    all_features = pd.concat([non_arrest_features, arrest_features], axis=1)
    all_features = all_features.join(arrest_temp[['hour', 'day_of_week', 'month', 'is_weekend', 
                                                'time_night', 'time_morning', 'time_afternoon', 'time_evening',
                                                'is_holiday']])
    
    # Get target for the specific grid
    y = target[f'{grid_to_predict}_target']
    
    # Align features and target
    all_features = all_features.loc[y.index]
    y = y.loc[all_features.index]
    
    # Remove NaN values
    mask = ~y.isna()
    all_features = all_features[mask]
    y = y[mask]
    
    # Feature selection - remove features with more than 50% missing values
    missing_ratio = all_features.isna().mean()
    all_features = all_features.drop(columns=missing_ratio[missing_ratio > 0.5].index)
    
    # Fill remaining missing values
    all_features = all_features.fillna(0)
    
    # 2. Train-Test Split (Time-based)
    print("Splitting data for training and testing...")
    train_size = int(len(all_features) * 0.8)
    X_train = all_features.iloc[:train_size]
    X_test = all_features.iloc[train_size:]
    y_train = y.iloc[:train_size]
    y_test = y.iloc[train_size:]
    print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    # print(X_train.dtypes)
    
    # 3. Model Training and Evaluation
    print("Training models...")
    models = {}
    # for model_type in ['xgboost', 'lightgbm', 'randomforest', 'logistic']:
    for model_type in ['lightgbm']:
        print(f"\nTraining {model_type} model...")
        model, y_pred_proba = train_evaluate_model(X_train, X_test, y_train, y_test, model_type)
        models[model_type] = model
    
    return models, all_features, y


# Execute the pipeline for a specific grid (e.g., downtown area)
# models, features, target = crime_prediction_pipeline('0_7')

# Plot the predictions over time for the best model
# best_model = models['lightgbm']
# y_pred_proba = best_model.predict_proba(features.iloc[int(len(features) * 0.8):])[:, 1]

# Create a time series plot with predictions
# plt.figure(figsize=(15, 8))
# plt.plot(target.iloc[int(len(target) * 0.8):].index, target.iloc[int(len(target) * 0.8):], label='Actual')
# plt.plot(target.iloc[int(len(target) * 0.8):].index, y_pred_proba, label='Predicted Probability')
# plt.title('Crime Prediction in Grid 5_5 (Downtown Chicago)')
# plt.xlabel('Time')
# plt.ylabel('Crime Occurrence Probability')
# plt.legend()
# plt.grid(True)
# plt.tight_layout()
# plt.show()

In [ ]:
# Modified function to prepare data for all grids
def prepare_all_grids_data():
    """
    Prepare data for all grids at once to train a single model
    """
    import warnings
    from pandas.errors import PerformanceWarning
    
    # Suppress PerformanceWarning
    warnings.filterwarnings("ignore", category=PerformanceWarning)
    
    print("Preparing prediction model for all grids...")
    
    # 1. Add temporal features
    print("Adding temporal features...")
    arrest_temp = add_temporal_features(arrest_data)
    non_arrest_temp = add_temporal_features(non_arrest_data)
    
    # Get all grid columns
    grid_columns = [col for col in non_arrest_data.columns if '_' in col][:2]
    
    print(f"Processing {len(grid_columns)} grids...")
    
    # 2. Create a combined dataframe for all grids
    all_data = []
    
    for grid in grid_columns:
        print(f"Processing grid {grid}...")
        
        # Extract data for this grid
        arrest_grid = arrest_temp[[grid]].rename(columns={grid: 'grid_value'})  # Rename to standard column name
        non_arrest_grid = non_arrest_temp[[grid]].rename(columns={grid: 'grid_value'})  # Rename to standard column name
        
        # Add rolling and lag features
        arrest_features = add_rolling_features(arrest_grid)
        arrest_features = add_lag_features(arrest_features)
        
        non_arrest_features = add_rolling_features(non_arrest_grid)
        non_arrest_features = add_lag_features(non_arrest_features)
        
        # Create target variable
        target = prepare_target(non_arrest_grid)
        
        # Combine features
        features = pd.concat([non_arrest_features, arrest_features], axis=1)
        features = features.join(arrest_temp[['hour', 'day_of_week', 'month', 'is_weekend', 
                                           'time_night', 'time_morning', 'time_afternoon', 'time_evening',
                                           'is_holiday']])
        
        # Get target for this grid
        y = target[f'target']
        
        # Align features and target
        features = features.loc[y.index]
        y = y.loc[features.index]
        
        # Remove NaN values
        mask = ~y.isna()
        features = features[mask]
        y = y[mask]
        
        # Add grid identifier
        features['grid_id'] = grid
        features['target'] = y.values
        
        # Reset index to avoid duplicate indices when concatenating
        features = features.reset_index()
        
        # Append to our collection
        all_data.append(features)
    
    print(len(all_data))
    for df in all_data:
        print(df.columns.tolist())
        print(df.head(3))
        print(df.index.is_unique)
    

    # Combine all grid data into one dataframe
    combined_data = pd.concat(all_data, axis=0, ignore_index=True)
    
    # Feature selection - remove features with more than 50% missing values
    missing_ratio = combined_data.isna().mean()
    combined_data = combined_data.drop(columns=missing_ratio[missing_ratio > 0.5].index)
    
    # Fill remaining missing values
    combined_data = combined_data.fillna(0)
    
    # Extract target
    y = combined_data['target']
    X = combined_data.drop(columns=['target'])
    
    return X, y

# Modified function to train model on all grids
def train_model_all_grids():
    """Train a single model on data from all grids."""
    import warnings
    from pandas.errors import PerformanceWarning
    
    # Suppress PerformanceWarning
    warnings.filterwarnings("ignore", category=PerformanceWarning)
    
    # Prepare data
    X, y = prepare_all_grids_data()
    
    # Time-based split
    print("Splitting data for training and testing...")
    train_size = int(len(X) * 0.8)
    X_train = X.iloc[:train_size]
    X_test = X.iloc[train_size:]
    y_train = y.iloc[:train_size]
    y_test = y.iloc[train_size:]
    
    print(f"Training data shapes: X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"Testing data shapes: X_test: {X_test.shape}, y_test: {y_test.shape}")
    
    # Handle different data types
    
    # 1. First identify timestamp/datetime columns and convert them properly
    for col in X_train.select_dtypes(include=['datetime64']).columns:
        # Extract useful features from timestamps instead of using raw timestamps
        X_train[f'{col}_hour'] = X_train[col].dt.hour
        X_train[f'{col}_day'] = X_train[col].dt.day
        X_train[f'{col}_month'] = X_train[col].dt.month
        X_train[f'{col}_year'] = X_train[col].dt.year
        
        X_test[f'{col}_hour'] = X_test[col].dt.hour
        X_test[f'{col}_day'] = X_test[col].dt.day
        X_test[f'{col}_month'] = X_test[col].dt.month
        X_test[f'{col}_year'] = X_test[col].dt.year
        
        # Drop the original timestamp column
        X_train = X_train.drop(columns=[col])
        X_test = X_test.drop(columns=[col])
    
    # 2. Convert any object columns to numeric
    for col in X_train.select_dtypes(include=['object']).columns:
        X_train[col] = pd.to_numeric(X_train[col], errors='coerce')
        X_test[col] = pd.to_numeric(X_test[col], errors='coerce')
    
    # 3. Handle index column if it exists
    if 'index' in X_train.columns:
        X_train = X_train.drop(columns=['index'])
        X_test = X_test.drop(columns=['index'])
    
    # 4. Convert grid_id to numeric if it's categorical
    if 'grid_id' in X_train.columns and X_train['grid_id'].dtype == 'object':
        # Create a label encoder for the grid_id
        from sklearn.preprocessing import LabelEncoder
        le = LabelEncoder()
        X_train['grid_id'] = le.fit_transform(X_train['grid_id'])
        X_test['grid_id'] = le.transform(X_test['grid_id'])
    
    # Fill NA values
    X_train = X_train.fillna(0)
    X_test = X_test.fillna(0)
    
    # Final check for numeric types
    for col in X_train.columns:
        if not np.issubdtype(X_train[col].dtype, np.number):
            print(f"Warning: Column {col} is not numeric, converting...")
            X_train[col] = pd.to_numeric(X_train[col], errors='coerce')
            X_test[col] = pd.to_numeric(X_test[col], errors='coerce')
    
    # Convert to numpy arrays
    X_train_np = X_train.values.astype(np.float32)
    X_test_np = X_test.values.astype(np.float32)
    
    # Train LightGBM model
    print("Training LightGBM model on all grids...")
    model = lgb.LGBMClassifier(
        learning_rate=0.1,
        n_estimators=200,
        max_depth=7,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )
    
    model.fit(X_train_np, y_train)
    
    # Evaluate model
    y_pred_proba = model.predict_proba(X_test_np)[:, 1]
    y_pred = model.predict(X_test_np)
    
    # Calculate metrics
    precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
    pr_auc = auc(recall, precision)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    print(f"ROC AUC: {roc_auc:.4f}")
    print(f"PR AUC: {pr_auc:.4f}")
    print(classification_report(y_test, y_pred))
    
    # Feature importance
    feature_importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("Top 10 important features:")
    print(feature_importance.head(10))
    
    return model, X_train, y_train

# Run the all-grids pipeline
model, X, y = train_model_all_grids()